# Assignment 2- TDT4265
#### Sara L. Ludvigsen and Emma H. Buøen
##### April 2020

## Task 1 - Softmax regression with backpropagation
### 1.a - Backpropagation
In this task we will show that 
$$
w_{ji} := w_{ji} - \alpha \delta_j x_i
$$
and that
$$
\delta_j = f'(z_j) \sum_k w_{kj} \delta_k .
$$

To do this, we first need to define our cost function as:
$$
C(w) = - \frac{1}{N} \sum_{n = 1}^{N} \sum_{k = 1}^{K} y^n_k \ln(\hat{y}^n_k)
$$

We know from the problem description that the activation of hidden unit $j$ as $a_j = f(z_j)$, where f is the sigmoid function. It is also stated that
$$
z_j = \sum_{i=0}^d w_{ji} x_i,
$$
and that $d$ is the dimensionality of the input. 

In the previous assignment, we showed that the update rule for for the weights $w_{kj}$ of the output
layer is
$$
w_{kj} := w_{kj} - \alpha \delta_k a_j, 
$$
where $\delta_k = \frac{\partial C}{\partial z_k} = -(y_k - \hat{y}_k)$.

Calculating the update rule for the weights $w_{ji}$:
$$
w_{ji} := w_{ji} - \alpha \frac{\partial C}{\partial w_{ij}} = w_{ji} - \alpha \overbrace{\frac{\partial C}{\partial z_j}}^{\delta_j} \overbrace{\frac{\partial z_j}{\partial w_{ij}}}^{x_i} = \underline{w_{ji} - \alpha \delta_j x_i}
$$
$$
\delta_j = \frac{\partial C}{\partial z_j} = \frac{\partial C}{\partial a_j}\frac{\partial a_j}{\partial z_j} = \sum_k \overbrace{\frac{\partial C}{\partial z_k}}^{\delta_k}  \overbrace{\frac{\partial z_k}{\partial a_j}}^{w_{kj}} \overbrace{\frac{\partial a_j}{\partial z_j}}^{f'(z_j)} = f'(z_j) \sum_k w_{kj} \delta_k
$$


### 1.b -  Vectorize computation